# Black Box Model 2: LSTM in PyTorch

In [3]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, Dataset, DataLoader
import lightning as L

pd.options.display.max_columns = 500

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


## Data Preparation

In [4]:
df = pd.read_csv("../00 Data Retrieval and Cleaning/0_df_final_imputed.csv")

# unselect the wrong direction target variable
df = df.drop(columns=["auction_price_ch_de", "allocatedCapacity_ch_de", "ATC_ch_de"])

# drop variables that are not needed
df = df.drop(columns=["ATC_de_ch", "allocatedCapacity_de_ch"])

In [5]:
df.head(3)

,date,auction_price_de_ch,dst,day_ahead_price_at,day_ahead_price_ch,day_ahead_price_de,day_ahead_price_fr,actual_load_at,actual_load_de,biomass_actual_aggregated_at,fossil_gas_actual_aggregated_at,fossil_hard_coal_actual_aggregated_at,hydro_pumped_storage_actual_aggregated_at,hydro_pumped_storage_actual_consumption_at,hydro_run-of-river_and_poundage_actual_aggregated_at,hydro_water_reservoir_actual_aggregated_at,solar_actual_aggregated_at,wind_onshore_actual_aggregated_at,biomass_actual_aggregated_de,fossil_brown_coal_lignite_actual_aggregated_de,fossil_gas_actual_aggregated_de,fossil_hard_coal_actual_aggregated_de,fossil_oil_actual_aggregated_de,geothermal_actual_aggregated_de,hydro_pumped_storage_actual_aggregated_de,hydro_pumped_storage_actual_consumption_de,hydro_run-of-river_and_poundage_actual_aggregated_de,hydro_water_reservoir_actual_aggregated_de,other_actual_aggregated_de,other_renewable_actual_aggregated_de,solar_actual_aggregated_de,waste_actual_aggregated_de,wind_offshore_actual_aggregated_de,wind_onshore_actual_aggregated_de,hydro_pumped_storage_actual_aggregated_fr,hydro_pumped_storage_actual_consumption_fr,actual_load_ch,actual_load_fr,actual_load_it,solar_forecast_de,wind_onshore_forecast_fr,hydro_pumped_storage_ch,hydro_run-of-river_and_poundage_ch,hydro_water_reservoir_ch,nuclear_ch,solar_ch,wind_onshore_ch,nuclear_actual_aggregated_de,biomass_actual_aggregated_fr,fossil_gas_actual_aggregated_fr,fossil_hard_coal_actual_aggregated_fr,fossil_oil_actual_aggregated_fr,hydro_run-of-river_and_poundage_actual_aggregated_fr,hydro_water_reservoir_actual_aggregated_fr,nuclear_actual_aggregated_fr,solar_actual_aggregated_fr,waste_actual_aggregated_fr,wind_onshore_actual_aggregated_fr,biomass_actual_aggregated_it,fossil_coal-derived_gas_actual_aggregated_it,fossil_gas_actual_aggregated_it,fossil_hard_coal_actual_aggregated_it,fossil_oil_actual_aggregated_it,geothermal_actual_aggregated_it,hydro_pumped_storage_actual_aggregated_it,hydro_pumped_storage_actual_consumption_it,hydro_run-of-river_and_poundage_actual_aggregated_it,hydro_water_reservoir_actual_aggregated_it,other_actual_aggregated_it,solar_actual_aggregated_it,waste_actual_aggregated_it,wind_onshore_actual_aggregated_it,hydro_reservoir_storage_at,hydro_reservoir_storage_ch,hydro_reservoir_storage_fr,hydro_reservoir_storage_it,crossborder_actual_flow_at_ch,crossborder_actual_flow_ch_at,crossborder_actual_flow_ch_de_lu,crossborder_actual_flow_ch_fr,crossborder_actual_flow_ch_it,crossborder_actual_flow_de_lu_ch,crossborder_actual_flow_fr_ch,crossborder_actual_flow_it_ch,capacity_forecast_at_ch,capacity_forecast_ch_at,capacity_forecast_ch_de_lu,capacity_forecast_ch_fr,capacity_forecast_ch_it,capacity_forecast_de_lu_ch,capacity_forecast_FR_CH,capacity_forecast_it_ch,actual_load_ch_missing_dummy,actual_load_fr_missing_dummy,actual_load_it_missing_dummy,solar_forecast_de_missing_dummy,wind_onshore_forecast_fr_missing_dummy,hydro_pumped_storage_ch_missing_dummy,hydro_run-of-river_and_poundage_ch_missing_dummy,hydro_water_reservoir_ch_missing_dummy,nuclear_ch_missing_dummy,solar_ch_missing_dummy,wind_onshore_ch_missing_dummy,nuclear_actual_aggregated_de_missing_dummy,biomass_actual_aggregated_fr_missing_dummy,fossil_gas_actual_aggregated_fr_missing_dummy,fossil_hard_coal_actual_aggregated_fr_missing_dummy,fossil_oil_actual_aggregated_fr_missing_dummy,hydro_run-of-river_and_poundage_actual_aggregated_fr_missing_dummy,hydro_water_reservoir_actual_aggregated_fr_missing_dummy,nuclear_actual_aggregated_fr_missing_dummy,solar_actual_aggregated_fr_missing_dummy,waste_actual_aggregated_fr_missing_dummy,wind_onshore_actual_aggregated_fr_missing_dummy,biomass_actual_aggregated_it_missing_dummy,fossil_coal-derived_gas_actual_aggregated_it_missing_dummy,fossil_gas_actual_aggregated_it_missing_dummy,fossil_hard_coal_actual_aggregated_it_missing_dummy,fossil_oil_actual_aggregated_it_missing_dummy,geothermal_actual_aggregated_it_missing_dummy,hydro_pumped_storage_actual_aggregated_it_

In [6]:
df.shape

(44510, 148)

In [7]:
X = df.drop(columns=["date", "auction_price_de_ch"])
y = df["auction_price_de_ch"]

train_size = int(0.6 * X.shape[0])
val_size = int(0.2 * X.shape[0])
test_size = X.shape[0] - train_size - val_size

sc = StandardScaler()

X_train = X.iloc[:train_size, :]
X_val = X.iloc[train_size:train_size + val_size, :]
X_test = X.iloc[train_size + val_size:, :]

y_train = y.iloc[:train_size].to_numpy()
y_val = y.iloc[train_size:train_size + val_size].to_numpy()
y_test = y.iloc[train_size + val_size:].to_numpy()

X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

## Model

In [8]:
class LightningLSTM(L.LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, output_size, lr=0.01,
                 noise_std=0.01):
        super().__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.noise_std = noise_std
        self.lr = lr

        # Define LSTM layer
        self.lstm = nn.LSTM(
            input_size=input_size,  # number of features in input data
            hidden_size=self.hidden_size,  # number of output values
            num_layers=self.num_layers,
            batch_first=True,
        )
        self.add_noise_to_weights(self.lstm, self.noise_std)

        # Define fully connected layer
        self.fc = nn.Linear(self.hidden_size, output_size)

    def add_noise_to_weights(self, layer, std):
        for param in layer.parameters():
            if param.requires_grad:
                param.data.add_(torch.randn(param.size()) * std)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        
        return out
    
    def configure_optimizers(self):
        return Adam(self.parameters(), self.lr)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)

        # Calculate evaluation metric
        loss = nn.functional.l1_loss(y_pred, y)
        eval_metric = mean_absolute_error(y_pred.detach().numpy(), y.detach().numpy())

        # Log the evaluation metric to the training progress bar
        self.log("train/train_eval_metric", eval_metric, on_step=True, on_epoch=False)

        return {"loss": loss, "eval_metric": eval_metric}
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        eval_metric = mean_absolute_error(y_pred.detach().numpy(), y.detach().numpy())
        
        # Log the evaluation metric to the validation progress bar
        self.log("val/val_eval_metric", eval_metric, on_step=True, on_epoch=False)
        
        return {"val_eval_metric": eval_metric}
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        eval_metric = mean_absolute_error(y_pred.detach().numpy(), y.detach().numpy())
        
        # Log the evaluation metric to the validation progress bar
        self.log("val/test_eval_metric", eval_metric, on_step=True, on_epoch=False)
        
        return {"test_eval_metric": eval_metric}


In [9]:
# Set the default data type for torch tensors to float32
torch.set_default_dtype(torch.float32)

In [10]:
# Create Sequences and Targets
def create_sequences(X, y, sequence_length, target_length):
    """Create sequences and targets from data.

    Args:
        X (np.ndarray): Input data.
        y (pd.Series): Target data.
        sequence_length (int): Length of the sequence.
        target_length (int): Length of the target/forecasting horizon in periods.

    Returns:
        _type_: _description_
    """
    sequences = []
    targets = []
    for i in tqdm(range(X.shape[0] - sequence_length - target_length + 1)):
        # seq = X[i:i + sequence_length, :]
        # target = y[i + sequence_length:i + sequence_length + target_length]
        seq = X[i:(i + sequence_length + target_length), :]
        target = y[(i + sequence_length):(i + sequence_length + target_length)]
        sequences.append(seq)
        targets.append(target)
    sequences = np.array(sequences)
    targets = np.array(targets)
    
    return torch.tensor(sequences, dtype = torch.float32), torch.tensor(targets, dtype = torch.float32)

In [11]:
X_train_seq, y_train_seq = create_sequences(X_train, y_train, 48, 24)
X_val_seq, y_val_seq = create_sequences(X_val, y_val, 48, 24)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, 48, 24)

  0%|          | 0/26635 [00:00<?, ?it/s]

  0%|          | 0/8831 [00:00<?, ?it/s]

  0%|          | 0/8831 [00:00<?, ?it/s]

In [12]:
class CustomDataset(Dataset):
    def __init__(self, inputs, targets):
        """
        Args:
            inputs (list): List of input sequences (PyTorch tensors or NumPy arrays).
            targets (list): List of target sequences (PyTorch tensors or NumPy arrays).
        """
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_sequence = self.inputs[idx]
        target_sequence = self.targets[idx]

        return input_sequence, target_sequence

In [13]:
X.shape

(44510, 146)

In [14]:
train_set = CustomDataset(X_train_seq, y_train_seq)
val_set = CustomDataset(X_val_seq, y_val_seq)
test_set = CustomDataset(X_test_seq, y_test_seq)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

In [15]:
# Create a Lightning model with the sampled hyperparameters
model = LightningLSTM(
    input_size=X.shape[1],
    hidden_size=128,
    num_layers=2,
    output_size=24,
    lr=0.01,
    noise_std=0.01,
)

model

LightningLSTM(
  (lstm): LSTM(146, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=24, bias=True)
)

In [16]:
# Create a PyTorch Lightning Trainer
trainer = L.Trainer(max_epochs=10)  # Adjust the number of epochs as needed

# Train the model
trainer.fit(model, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\mathi\miniconda3\envs\statslab\lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Missing logger folder: c:\Users\mathi\OneDrive\Universitaet\2nd Semester\2_Statistics Lab\RWE-ETH-Power-Trading\01 Task 1 - Spot Price\lightning_logs

  | Name | Type   | Params
--------------------------------
0 | lstm | LSTM   | 273 K 
1 | fc   | Linear | 3.1 K 
--------------------------------
276 K     Tra

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\mathi\miniconda3\envs\statslab\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\mathi\miniconda3\envs\statslab\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

c:\Users\mathi\miniconda3\envs\statslab\lib\site-packages\lightning\pytorch\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [17]:
# Return the metric you want to optimize (e.g., validation loss)
trainer.logged_metrics.get("val/val_eval_metric").item()

2.857717752456665

### Making Predictions on the Test Period

In [24]:
trainer.predict(model=model, dataloaders=train_loader)

c:\Users\mathi\miniconda3\envs\statslab\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:492: Your `predict_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
c:\Users\mathi\miniconda3\envs\statslab\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

AttributeError: 'list' object has no attribute 'size'